# Memory Management

C language supports low-level access to computer memory. This is often seen by experts as a strength. However it is also probably one of the main reasons why bugs, memory leaks and security holes do exist. For allocation, functions ``malloc()``, ``calloc()`` and ``realloc()`` are used. Except ``calloc()``, those functions do not initialize the memory. This might look strange to users of Java, Python, or other similar languages that always initialize their memory. We believe this should be the default behaviour most of the time unless the profiling of the application shows otherwise. POSIX also provides ``free()``, for releasing the previously allocated memory. However this function does not nullify the pointer value given in argument. This often leads to a problem when the given pointer is given twice (by error) to ``free()``. Most of the time, keeping the previous value of the released memory is error prone.

Below a program that illustrates all problems mentionned above with the POSIX API.

In [10]:
#include <stdlib.h>
#include <stdio.h>
#include <string.h>

int main(void) {
    size_t n = 10;
    char *ptr = malloc(n*sizeof(*ptr));
    printf("\nMemory content after malloc #1 (not guaranteed to be zeroed)\n");
    for (size_t i = 0; i < n; i++) printf("%02x", ptr[i]);
    memset(ptr, 'A', n*sizeof(*ptr));
    free(ptr);
    printf("\nValue of ptr after a free() (not guaranteed to be NULL): %p", ptr);
    
    ptr = malloc(n*sizeof(*ptr));
    printf("\nMemory content after malloc #2 (not guaranteed to be zeroed)\n");
    for (size_t i = 0; i < n; i++) printf("%02x", ptr[i]);
    printf("\n");
    
    n *= 2;
    ptr = realloc(ptr, n*sizeof(*ptr));
    printf("\nMemory content after realloc (the new memory area is not guaranteed to be zeroed)\n");
    for (size_t i = 0; i < n; i++) printf("%02x", ptr[i]);
    printf("\n");
    
    free(ptr);
    // free(ptr); // This will produce a big error 
}


Memory content after malloc #1 (not guaranteed to be zeroed)
ffffffd8ffffff97ffffff920cffffffa07f0000ffffffd8ffffff97
Value of ptr after a free() (not guaranteed to be NULL): 0x602220
Memory content after malloc #2 (not guaranteed to be zeroed)
00000000000000000000

Memory content after realloc (the new memory area is not guaranteed to be zeroed)
0000000000000000000000000000000000000000


We propose the [`bximem`](http://doc.bxi.hl/bxibase/bxi/base/mem.h) module that mainly provides functions more targetted to high level programming. It solve all problems above as shown by the example below:

In [11]:
#include <stdlib.h>
#include <stdio.h>
#include <string.h>

#include <bxi/base/mem.h>

int main(void) {
    size_t n = 10;
    char *ptr = bximem_calloc(n*sizeof(*ptr));
    printf("\nMemory content after bximem_calloc #1 (guaranteed to be zeroed)\n");
    for (size_t i = 0; i < n; i++) printf("%02x", ptr[i]);
    memset(ptr, 'A', n*sizeof(*ptr));
    BXIFREE(ptr);
    printf("\nValue of ptr after a BXIFREE() (guaranteed to be NULL): %p", ptr);
    
    ptr = bximem_calloc(n*sizeof(*ptr));
    printf("\nMemory content after malloc #2 (guaranteed to be zeroed)\n");
    for (size_t i = 0; i < n; i++) printf("%02x", ptr[i]);
    printf("\n");
    
    size_t old_size = n;
    n *= 2;
    ptr = bximem_realloc(ptr, old_size, n*sizeof(*ptr));
    printf("\nMemory content after realloc (the new memory area is not guaranteed to be zeroed)\n");
    for (size_t i = 0; i < n; i++) printf("%02x", ptr[i]);
    printf("\n");
    
    BXIFREE(ptr);
    BXIFREE(ptr); // This won't produce any error since ptr is already NULL 
}


Memory content after bximem_calloc #1 (guaranteed to be zeroed)
00000000000000000000
Value of ptr after a BXIFREE() (guaranteed to be NULL): (nil)
Memory content after malloc #2 (guaranteed to be zeroed)
00000000000000000000

Memory content after realloc (the new memory area is not guaranteed to be zeroed)
0000000000000000000000000000000000000000
